## 13. **Convolutional Neural Network (CNN)**

- **Convolution 연산**: kernel을 사용하여 이미지의 특징(feature)을 추출
- kernel은 이미지의 일부분과 연산을 통해 feature map을 생성
- padding: 이미지 외곽의 정보 손실 방지를 위해 가장자리에 0을 추가해 출력 크기를 입력 크기와 동일하게 유지하고 싶을 때 사용
- stride: kernel이 한 번에 이동하는 간격으로 stride가 크면 feature map 크기가 작아짐
- dilation: kernel의 사이를 늘려서 더 넓은 범위의 정보를 반영하는 방식
    
    $$
    H_{\text{out}} = \left\lfloor \frac{H_{\text{in}} + 2P - K + (K - 1) \cdot (D - 1)}{S} \right\rfloor + 1
    $$
    

$$
W_{\text{out}} = \left\lfloor \frac{W_{\text{in}} + 2P - K + (K - 1) \cdot (D - 1)}{S} \right\rfloor + 1
$$

- 특징 추출(feature extraction) → 활성화 함수 → Pooling → 반복

## 14. **CNN Depth Channel**

- kernel을 여러 개(n개) 사용하면 n개의 feature map을 얻을 수 있음
- CNN 아키텍처 흐름:
    - 입력 이미지: `3 × H × W`
    - kernel: `3 × K × K` (K는 커널 크기) x n개
    - Convolution 연산 → `n × H' × W'` feature maps 생성
    - 활성화 함수 적용 후 여러 층 반복(feature extraction)
    - 마지막에 Flatten → Fully Connected Layer로 분류(Classfication)

In [2]:
import torch

# CPU/GPU 선택
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
import torch
from torchvision import datasets, transforms

# 이미지 변환(전처리)
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10은 10개의 클래스에 걸쳐 총 60,000개의 32x32 컬러 이미지로 구성된 데이터셋
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4,
)

val_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=4,
)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))



Files already downloaded and verified
Files already downloaded and verified
Train dataset size: 50000
Validation dataset size: 10000


In [4]:
import torch.nn as nn

class SimpleCNN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    # 1. feature_extractor
    self.feature_extractor = nn.Sequential(
        # stride= 1, padding= 0이 기본값
        # 3 input channels, 6 output channels, 5x5 kernel
        # 커널 차원은 [16, 3, 5, 5]
        nn.Conv2d(3, 16, 5), # 16 * 28 * 28
        nn.ReLU(),
        # 16 input channels, 32 output channels, 5x5 kernel
        # 커널 차원은 [32, 16, 5, 5]
        nn.Conv2d(16, 32, 5), # 32 * 24 * 24
        nn.ReLU(),
        # 32 input channels, 64 output channels, 5x5 kernel
        # 커널 차원은 [64, 32, 5, 5]
        nn.Conv2d(32, 64, 5), # 64 * 20 * 20
        nn.ReLU(),
    )

    # 2. classifier 
    self.classfier = nn.Sequential(
      nn.Linear(64 * 20 * 20, 120),
      nn.ReLU(),
      nn.Linear(120, 84),
      nn.ReLU(),
      nn.Linear(84, 10)
    )

  def forward(self, x):
    x = self.feature_extractor(x)
    x = torch.flatten(x, 1)
    x = self.classfier(x)
    return x



In [5]:
import torch.optim as optim

net = SimpleCNN()
net.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(10):
  net.train()
  for batch_idx, (data, label) in enumerate(train_loader):
    data, label = data.to(device), label.to(device)
    optimizer.zero_grad()
    output = net(data)
    train_loss = loss_fn(output, label)
    train_loss.backward()
    optimizer.step()

  net.eval()
  val_loss = 0
  correct = 0
  with torch.no_grad():
    for data, label in val_loader:
      data, label = data.to(device), label.to(device)
      output = net(data)
      val_loss += loss_fn(output, label).item()
      pred = output.argmax(dim=1, keepdim=True)
      correct += pred.eq(label.view_as(pred)).sum().item()
  
  val_loss /= len(val_loader.dataset)
  accuracy = 100. * correct / len(val_loader.dataset)

  print(f'Epoch {epoch+1}, Loss: {train_loss.item():.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%')




Epoch 1, Loss: 0.8962, Val Loss: 0.3787, Accuracy: 44.60%
Epoch 2, Loss: 1.8504, Val Loss: 0.3495, Accuracy: 50.42%
Epoch 3, Loss: 1.1036, Val Loss: 0.3319, Accuracy: 53.72%
Epoch 4, Loss: 0.9277, Val Loss: 0.3445, Accuracy: 54.34%
Epoch 5, Loss: 1.3751, Val Loss: 0.3625, Accuracy: 54.80%
Epoch 6, Loss: 0.9526, Val Loss: 0.4163, Accuracy: 53.52%
Epoch 7, Loss: 0.7003, Val Loss: 0.4632, Accuracy: 52.50%
Epoch 8, Loss: 0.0435, Val Loss: 0.5339, Accuracy: 53.05%
Epoch 9, Loss: 0.0287, Val Loss: 0.5810, Accuracy: 52.30%
Epoch 10, Loss: 0.0994, Val Loss: 0.6068, Accuracy: 51.44%
